# Math Question Answer Verification Competition

## Starter Code

Borrowed from [official Unsloth implementation](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=MKX_XKs_BNZR)

In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [3]:
import torch
import pandas as pd
from unsloth import FastLanguageModel
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Load Model and Wrap with LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Competition Dataset

In [6]:
# download and load competition dataset
from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")

README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
prompt = """You are a great mathematician tasked with determining if a math question's answer is correct. Based on the question, user's answer, and user's explanation, provide only "True" or "False" as output.

### Question:
{}

### User's Answer:
{}

### User's Explanation:
{}

### Output (Answer with one word: "True" or "False"):
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []

    for instruction, input, explanation, output in zip(questions, answers, solutions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, explanation, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Process the training dataset and generate prompt for each datapoint
train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

## SFT

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 7,
    warmup_steps = 5,
    max_steps = 200,
    # num_train_epochs = 1, # Set this for 1 full training run.
    learning_rate = 3.5e-4,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    report_to = "none" # Use this for WandB etc
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

trainer_stats = trainer.train()

Map (num_proc=4):   0%|          | 0/1000000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 7
\        /    Total batch size = 28 | Total steps = 200
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,1.497500
2,1.556500
3,1.356100
4,1.053900
5,0.924700
6,0.773300
7,0.830400
8,0.782000
9,0.737700
10,0.793500


## Saving Model

In [ ]:
model_name = "model"
model.save_pretrained(model_name) # Local saving
tokenizer.save_pretrained(model_name)
torch.cuda.empty_cache() # Clear GPU cache

## Inference

In [ ]:
# Prepare to store results
predictions = []
test_dataset = dataset['test']

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "auto",  # Distributes layers across available GPUs
    offload_buffers=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
batch_size = 8

# Generate predictions for each batch in the test set
for i in range(0, len(test_dataset), batch_size):
    # Batch the questions, answers, and explanations
    batch_questions = test_dataset['question'][i:i + batch_size]
    batch_answers = test_dataset['answer'][i:i + batch_size]
    batch_solutions = test_dataset['solution'][i:i + batch_size]
    batch_ids = range(i, i + len(batch_questions))

    # Create input prompts for the batch
    input_prompts = [prompt.format(q, a, s, '') for q, a, s in zip(batch_questions, batch_answers, batch_solutions)]

    # Tokenize the batch of inputs
    inputs = tokenizer(input_prompts, return_tensors="pt", padding=True).to(device)

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Process each output in the batch
    for j, output in enumerate(outputs):
        input_token_len = len(inputs['input_ids'][j])  # Get token length for each batch item
        response = tokenizer.decode(output[input_token_len:], skip_special_tokens=True).strip()

        # Append the question ID and prediction to the list
        predictions.append({
            "ID": batch_ids[j],          # Generated question ID
            "is_correct": response  # Model's prediction, expected to be 'True' or 'False'
        })

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
failed_log = '/failed.txt'
file = open(failed_log, 'w')
for i in range(len(predictions)):
  if predictions[i]['is_correct'] not in ('True', 'False'):
    file.write(f"question {i}: {predictions[i]['is_correct']}\n")
file.close()

In [ ]:
# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions)

import re

# Define a function to clean and extract 'True' or 'False' from the response
def extract_boolean(response):
    # Use a regular expression to find all standalone "True" or "False" occurrences
    matches = re.findall(r'\b(True|False)\b', response)

    # Return the last occurrence if matches are found, otherwise return None
    return matches[-1] if matches else None

# Apply the function to the 'is_correct' column
predictions_df['is_correct'] = predictions_df['is_correct'].apply(extract_boolean)
predictions_df['is_correct'].fillna('False', inplace=True)

# Convert extracted strings to boolean values
predictions_df['is_correct'] = predictions_df['is_correct'].map({'True': True, 'False': False})

# Save the predictions to a CSV file
output_csv_path = "/submissions.csv"
predictions_df.to_csv(output_csv_path, index=False)


<ipython-input-11-5d563bb2fdf1>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  predictions_df['is_correct'].fillna('False', inplace=True)


In [ ]:
!zip -r /content/model.zip /content/model
from google.colab import files
files.download('/content/model.zip')
files.download(output_csv_path)
files.download(failed_log)

  adding: content/model/ (stored 0%)
  adding: content/model/tokenizer.json (deflated 85%)
  adding: content/model/special_tokens_map.json (deflated 71%)
  adding: content/model/adapter_model.safetensors (deflated 7%)
  adding: content/model/adapter_config.json (deflated 53%)
  adding: content/model/tokenizer_config.json (deflated 96%)
  adding: content/model/README.md (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>